## Webscraping
Iniciaremos o projeto realizando a raspagem da web, processo automatizado de extração de dados da web, no qual coletaremos as notícias relacionadas ao Liberation Day e às tarifas impostas pelo presidente Donald Trump divulgadas pelos principais veículos de notícias, como Brasil de Fato e Agência Brasil.

In [ ]:
#Primeiro, precisamos instalar algumas bibliotecas 
#%pip install pandas numpy requests bs4 scrapy tqdm


[notice] A new release of pip is available: 24.1.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Agora, precisamos importamos as bibliotecas para o arquivo
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import scrapy
import random
import time
from urllib.parse import urljoin
import re
from tqdm import tqdm

### Extração Automatizada de URLs via Web Scraping no Google
Aqui precisamos o código desenvolvido tem como objetivo automatizar a busca de sites oficiais ou relevantes relacionados a uma lista de nomes, utilizando técnicas de web scraping com o Google. A lógica é dividida em duas funções principais: a primeira, get_url_from_google, realiza uma busca no Google baseada no nome de uma entidade presente em um DataFrame (base), simulando o acesso de um navegador real para evitar bloqueios. Ela analisa os resultados da busca e tenta extrair a URL mais apropriada com base em padrões definidos. A segunda função, pega_url_de_lista_com_timesleep, aplica esse processo para um intervalo de índices da base de dados, realizando pausas aleatórias entre as requisições para respeitar os limites dos servidores e evitar bloqueios por excesso de acessos. O resultado final é um dicionário associando cada nome a uma ou mais URLs encontradas, o que permite enriquecer os dados com links úteis de forma automatizada, segura e eficiente.

## 1º Jornal Online - Agência Brasil

In [2]:
def get_url_from_agencia_brasil(index):
	

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		for i in range(1, 50):
				
			search_url = f"https://busca.ebc.com.br/nodes?page={i}utf8=%E2%9C%93&q={query}&site_id=agenciabrasil"
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			#print(links)
			lista.extend(links)
		# Break se não achar mais links (opcional)
			if not links:
				break

		return lista
	
	def encontra_links_do_html(soup):
		anchors = soup.find_all("a")
		links_encontrados = set()  # usando set para evitar duplicatas

		for tag in anchors:
			href = tag.get("href")
			if href and href.startswith("http://agenciabrasil"):
				links_encontrados.add(href.strip())

		return list(links_encontrados)
			
	return main(index)

def pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave):
	fim = dict()
	for chave in palavras_chave:
		url = get_url_from_agencia_brasil(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	return fim

palavras_chave = ['presidente', 'China', 'eua', 'Trump']

a = pega_url_de_lista_com_timesleep_agencia_brasil(palavras_chave)
a

NameError: name 'random' is not defined

In [ ]:
def extrai_dados_noticia(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
        "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
    ]

    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "pt-BR,pt;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")

    # Título
    titulo_tag = soup.find("h1")
    print(titulo_tag)
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Título não encontrado"

    # Data de publicação atualizada
    data_tag = soup.find("div", class_="data")
    if data_tag:
        data_texto = data_tag.get_text(strip=True)
        data_publicacao = data_texto.replace("Publicado em", "").strip()
    else:
        data_publicacao = "Data não encontrada"

    # Conteúdo principal atualizado
    conteudo_tag = soup.find("div", class_="conteudo-noticia")
    if conteudo_tag:
        paragrafos = conteudo_tag.find_all("p")
        conteudo = "\n".join(p.get_text(strip=True) for p in paragrafos if p.get_text(strip=True))
    else:
        conteudo = "Conteúdo não encontrado"

    return {
        "titulo": titulo,
        "data_publicacao": data_publicacao,
        "conteudo": conteudo
    }



In [ ]:
extrai_dados_noticia("https://agenciabrasil.ebc.com.br/direitos-humanos/noticia/2025-04/fruto-da-mobilizacao-indigena-apib-completa-20-anos-de-existencia")

In [ ]:
### Estrutura os dados em dicionários ###
b_dicionario = dict()
for b in list(a.keys()):
    c_dicionario = dict()
    for c in list(a.values())[0]:
        c_dicionario[c] = extrai_dados_noticia(c)
    b_dicionario[b] = c_dicionario


In [ ]:
# Transformar b_dicionario em uma lista de linhas para o DataFrame
linhas = []
for chave_principal, sub_dicionario in b_dicionario.items():
    for url, dados in sub_dicionario.items():
        linha = {
            "Palavra-chave": chave_principal,
            "URL": url,
            "Título": dados.get("titulo", "Título não encontrado"),
            "Data de Publicação": dados.get("data_publicacao", "Data não encontrada"),
            "Conteúdo": dados.get("conteudo", "Conteúdo não encontrado"),
        }
        linhas.append(linha)

# Criar o DataFrame
df = pd.DataFrame(linhas)

# Exibir o DataFrame
print(df)

In [ ]:
### Impostação para o Excel ###

import openpyxl
#df.to_excel("noticias_agencia_brasil.xlsx", index=False, engine='openpyxl')

## 2º Jornal Online - Brasil de Fato

In [12]:
def get_url_from_brasil_de_fato(index):
	

	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
			"Accept-Encoding": "gzip, deflate",  # tira o 'br' pois ele estava dando problema na formatação
			"Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "".join(index.split())
		lista = []
		for i in range(1, 22):	
			search_url = f"https://www.brasildefato.com.br/page/{i}/?s={query}"
			response = requests.get(search_url, headers=headers)
			response.encoding = response.apparent_encoding
			soup = BeautifulSoup(response.text, "html.parser")
			links = encontra_links_do_html(soup)
			lista.extend(links) 
		
		return lista
	

	def encontra_links_do_html(soup):
		main_data = soup.find_all("a", href=True)  # pega só <a> com href
		link_sites_encontrados = []

		for link in tqdm(main_data, desc="Links"):
			href = link['href']
			if href.startswith("https://www.brasildefato.com.br/2025") and href not in link_sites_encontrados:
				link_sites_encontrados.append(href)

		return link_sites_encontrados	
			
	return main(index)

def pega_url_de_lista_com_timesleep_brasil_de_fato(palavras_chave):
	fim = dict()
	for chave in tqdm(palavras_chave, desc="Scraping"):
		url = get_url_from_brasil_de_fato(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	
	return fim

palavras_chave = ['tarifas+trump']

d = pega_url_de_lista_com_timesleep_brasil_de_fato(palavras_chave)
print(d)


Links: 100%|█████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 177563.22it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████████████████| 180/180 [00:00<?, ?it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████████████████| 179/179 [00:00<?, ?it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 180432.69it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 179/179 [00:00<00:00, 179698.52it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 181/181 [00:00<00:00, 180025.85it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████████████████| 179/179 [00:00<?, ?it/s]

Links: 100%|█████████████████████████████████████████████████████████████████████| 159/159 [00:00<00:00, 159239.34it/s]

Links: 100%|████████████████████

{'tarifas+trump': ['https://www.brasildefato.com.br/2025/04/14/guerra-comercial-russia-tem-riscos-e-oportunidades-com-tarifas-de-trump/', 'https://www.brasildefato.com.br/2025/04/13/china-pede-fim-de-tarifas-reciprocas-de-trump-eua-dao-prazo-para-inicio-de-taxas-sobre-semicondutores/', 'https://www.brasildefato.com.br/2025/04/10/venezuela-aprova-decreto-de-emergencia-economica-para-combater-tarifas-de-trump/', 'https://www.brasildefato.com.br/2025/04/09/lula-critica-tarifas-arbitrarias-de-trump-em-discurso-na-celac-guerras-comerciais-nao-tem-vencedores/', 'https://www.brasildefato.com.br/2025/04/08/nao-aceitaremos-ameacas-afirma-china-em-resposta-a-intimidacao-de-trump-com-tarifas-adicionais-de-50/', 'https://www.brasildefato.com.br/2025/04/07/tarifas-de-trump-nao-reduzirao-deficit-de-balanca-comercial-dos-eua-diz-economista-chines/', 'https://www.brasildefato.com.br/2025/04/07/trump-ameaca-a-china-com-tarifas-extras-de-50-se-o-pais-nao-retirar-as-taxas-retaliatorias-de-34/', 'https://

In [44]:
def extrai_dados_noticia_brasil_de_fato(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
        "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
        "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
    ]

    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "pt-BR,pt;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")

    # Título
    titulo_tag = soup.find("h1")
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Título não encontrado"

    # Data de publicação atualizada
    data_tag = soup.find("div", class_="jeg_meta_date")
    if data_tag:
        data_texto = data_tag.get_text(strip=True)
        data_publicacao = data_texto.replace("Publicado em", "").strip()
    else:
        data_publicacao = "Data não encontrada"

    # Conteúdo principal atualizado
    conteudo_tag = soup.find("div", class_="content-inner")
    if conteudo_tag:
        paragrafos = conteudo_tag.find_all("p")
        conteudo = "\n".join(p.get_text(strip=True) for p in paragrafos if p.get_text(strip=True))
    else:
        conteudo = "Conteúdo não encontrado"

    return {
        "titulo": titulo,
        "data_publicacao": data_publicacao,
        "conteudo": conteudo
    }

In [41]:
extrai_dados_noticia_brasil_de_fato("https://www.brasildefato.com.br/2025/04/22/papa-francisco-acompanhou-de-perto-os-desdobramentos-do-crime-da-vale-em-brumadinho-mg/")

{'titulo': 'Papa Francisco acompanhou de perto os desdobramentos do crime da Vale em Brumadinho (MG)',
 'data_publicacao': '22.abr.2025 às 18h15',
 'conteudo': 'A morte do Papa Francisco, o primeiro nascido na América Latina, na segunda-feira (21), aos 88 anos de idade, emocionou milhões de seguidores e não seguidores da Igreja Católica. O fim do pontificado do jesuíta argentino fez com que seus apoiadores relembrassem alguns de seus feitos marcantes. Entre eles, ganharam destaque as homenagens do pontífice aos atingidos pelo rompimento da barragem da Vale em Brumadinho, Minas Gerais.\nEm 2019, logo após o crime que deixou 272 vítimas fatais em janeiro daquele ano, Francisco enviou um representante do Vaticano ao município da Região Metropolitana de Belo Horizonte (RMBH). Além disso, acompanhou de perto os desdobramentos e recebeu pessoas atingidas pelo rompimento.\nNos anos seguintes, o pontífice continuou relembrando os impactos da tragédia-crime.\n“Neste aniversário de um ano da tra

In [46]:
### Estrutura os dados em dicionários ###
e_dicionario = dict()
for e in tqdm(list(d.keys()), desc="Chave"):
    f_dicionario = dict()
    for f in list(d.values())[0]:
        print(f"Extraindo dados de: {f}")
        f_dicionario[f] = extrai_dados_noticia_brasil_de_fato(f)
    e_dicionario[e] = f_dicionario

Chave:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]

Extraindo dados de: https://www.brasildefato.com.br/2025/04/14/guerra-comercial-russia-tem-riscos-e-oportunidades-com-tarifas-de-trump/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/13/china-pede-fim-de-tarifas-reciprocas-de-trump-eua-dao-prazo-para-inicio-de-taxas-sobre-semicondutores/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/10/venezuela-aprova-decreto-de-emergencia-economica-para-combater-tarifas-de-trump/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/09/lula-critica-tarifas-arbitrarias-de-trump-em-discurso-na-celac-guerras-comerciais-nao-tem-vencedores/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/08/nao-aceitaremos-ameacas-afirma-china-em-resposta-a-intimidacao-de-trump-com-tarifas-adicionais-de-50/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/07/tarifas-de-trump-nao-reduzirao-deficit-de-balanca-comercial-dos-eua-diz-economista-chines/
Extraindo dados de: https://www.brasildefato.com.br/2025/04/07/trump-a

Chave: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [04:47<00:00, 287.78s/it]


In [47]:
# Transformar e_dicionario em uma lista de linhas para o DataFrame
linhas = []
for chave_principal, sub_dicionario in e_dicionario.items():
    for url, dados in sub_dicionario.items():
        linha = {
            "Palavra-chave": chave_principal,
            "URL": url,
            "Título": dados.get("titulo", "Título não encontrado"),
            "Data de Publicação": dados.get("data_publicacao", "Data não encontrada"),
            "Conteúdo": dados.get("conteudo", "Conteúdo não encontrado"),
        }
        linhas.append(linha)

# Criar o DataFrame
df = pd.DataFrame(linhas)

# Exibir o DataFrame
print(df)

     Palavra-chave                                                URL  \
0    tarifas+trump  https://www.brasildefato.com.br/2025/04/14/gue...   
1    tarifas+trump  https://www.brasildefato.com.br/2025/04/13/chi...   
2    tarifas+trump  https://www.brasildefato.com.br/2025/04/10/ven...   
3    tarifas+trump  https://www.brasildefato.com.br/2025/04/09/lul...   
4    tarifas+trump  https://www.brasildefato.com.br/2025/04/08/nao...   
..             ...                                                ...   
141  tarifas+trump  https://www.brasildefato.com.br/2025/02/04/dan...   
142  tarifas+trump  https://www.brasildefato.com.br/2025/02/03/tal...   
143  tarifas+trump  https://www.brasildefato.com.br/2025/01/26/col...   
144  tarifas+trump  https://www.brasildefato.com.br/2025/01/23/bra...   
145  tarifas+trump  https://www.brasildefato.com.br/2025/01/13/chi...   

                                                Título  \
0    Guerra comercial: Rússia tem riscos e oportuni...   
1    Ch

In [48]:
### Impostação para o Excel ###

import openpyxl
df.to_excel("noticias_brasil_de_fato.xlsx", index=False, engine='openpyxl')

## 3º Jornal Online - NPR ()

In [ ]:
def get_url_from_the_Atlantic(index):
	
	# Configuração de headers para simular um navegador
	user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Apple M1; Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.2277.128",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36 OPR/98.0.4759.82",
    "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; Pixel 7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; SM-S911B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; ONEPLUS A6013) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.107 Mobile Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 15393.44.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
    "Mozilla/5.0 (Android 14; Mobile; rv:124.0) Gecko/124.0 Firefox/124.0",
	]

	headers = {"User-Agent": random.choice(user_agents),
			"Accept-Language": "pt-BR,pt;q=0.9",
                "Connection": "keep-alive"
            }

	# Fazer a requisição ao Google
	def main(index):
		query = "+".join(index.split())
		lista = []
		
		search_url = f"https://www.theatlantic.com/search/?search-sort=relevance&q={query}&after=eyJjdXJzb3IiOls3OS4xMjY3LCI2ODI0MTkiXSwidGltZSI6IjIwMjUtMDQtMjBUMjE6MDQ6MjcuMjUwWiJ9&page=1"
		response = requests.get(search_url, headers=headers)
		soup = BeautifulSoup(response.text, "html.parser")
		#print(soup)
		links = encontra_links_do_html(soup)
		#print(links)
		lista.extend(links)
	# Break se não achar mais links (opcional)

		def scrape_page(url):
			response = requests.get(url, headers=headers)
			soup = BeautifulSoup(response.content, 'html.parser')

			# Procura o botão/link de próxima página
			next_page_tag = soup.find('a', {'data-event-element': 'more'})
			print(next_page_tag)
			if next_page_tag and 'href' in next_page_tag.attrs:
				next_url = urljoin('https://www.theatlantic.com', next_page_tag['href'])
				return next_url
			return None
		
		# Loop até não haver próxima página
		current_url = search_url
		i  = 0
		while i < 50:
			current_url = scrape_page(current_url)
			response = requests.get(search_url, headers=headers)
			soup = BeautifulSoup(response.text, "html.parser")
			#print(soup)
			links = encontra_links_do_html(soup)
			#print(links)
			lista.extend(links)
			i += 1
		# Break se não achar mais links (opcional)
			if not links:
				break
	

		return lista


	def encontra_links_do_html(soup):
		anchors = soup.find_all("a")
		links_encontrados = set()  # usando set para evitar duplicatas
		#print(anchors)
		for tag in anchors:
			href = tag.get("href")
			if (href and href.startswith("https://www.theatlantic.com/economy")) or (href and href.startswith("https://www.theatlantic.com/politics")) or (href and href.startswith("https://www.theatlantic.com/technology")) or (href and href.startswith("https://www.theatlantic.com/planet")) or (href and href.startswith("https://www.theatlantic.com/global")):
				links_encontrados.add(href.strip())

		return list(links_encontrados)
			
	return main(index)

def pega_url_de_lista_com_timesleep_the_Atlantic(palavras_chave):
	fim = dict()
	for chave in palavras_chave:
		url = get_url_from_the_Atlantic(chave)
		#print(f"URL encontrada para '{site}': {url}")
		fim[chave] = url
		# Pausa aleatória entre as requisições
		time.sleep(random.uniform(3, 15))
	return fim

palavras_chave = ['China and Trump', 'Tariffs', 'Europa and Trump']

a = pega_url_de_lista_com_timesleep_the_Atlantic(palavras_chave)
a

In [1]:
unique_urls = list(set(list(a.values())[2]))
unique_urls

NameError: name 'a' is not defined

In [ ]:
len(list(list(a.values())[0]))